<a href="https://colab.research.google.com/github/CoAxLab/Data-Explorations/blob/main/book/exercises/cross-validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 12: Cross validation
-----

In this exercise, we'll practice implementing cross validation techniques, including leave-one-out and k-fold cross validation. We'll use the `PimaIndiansDiabetes2` practice dataset, which has medical data on a group of Pima Native American women, including whether or not they have diabetes. This dataset is part of the `mlbench` package. We'll be using each person's medical history to predict whether or not they have been diagnosed with diabetes.

1. Data 1/1
2. Leave-one-out Cross Validation 4/4
3. Compare to cv.glm 3/3
4. Adjusting K and Reflection 2/2

# 1: Data (1 pts)
---

Load the `tidyverse`, `boot`, and `mlbench` packages (you may need to install `boot` and `mlbench`).

Load the `PimaIndiansDiabetes2` dataset using the `data()` function. Drop the `insulin` column (it just has a lot of missing data) and then drop `NA`s from the rest of the dataset. Save your updated dataset to a new variable name. Finally, print the dimensions of your new dataset, and look at the first few lines of data.

In [ ]:
# INSERT CODE HERE
#install.packages("boot")
#install.packages("mlbench")

library(tidyverse)
library(boot)
library(mlbench)
data(PimaIndiansDiabetes2)
df = PimaIndiansDiabetes2 %>%
  select(-c(insulin)) %>%
  drop_na()
head(df)
dim(df)


,pregnant,glucose,pressure,triceps,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,148,72,35,33.6,0.627,50,pos
2,1,85,66,29,26.6,0.351,31,neg
4,1,89,66,23,28.1,0.167,21,neg
5,0,137,40,35,43.1,2.288,33,pos
7,3,78,50,32,31.0,0.248,26,pos
9,2,197,70,45,30.5,0.158,53,pos


[1] 532   8

(Note that in medical contexts, `pedigree` refers to a system of measuring family history of a condition. So here, higher numbers mean greater family history of diabetes. You can read more about this dataset [here](https://rdrr.io/cran/mlbench/man/PimaIndiansDiabetes.html).)

# 2. Leave-one-out Cross Validation (4 pts)

In the tutorial, we learned how to fit leave-one-out cross validation using the `cv.glm` function from the `boot` package. But we can also do this manually using `predict()` like we have in the past.

Let's predict `diabetes`, a dichotomous outcome, using all the other variables in our modified dataset.

First, fit a logistic regression model using all of the observations except the very first one. Then use your fitted model to predict whether your holdout case is positive or negative for diabetes. Remember that logistic regression coefficients are in **log-odds**, meaning that if an output is positive, the probability of the outcome is greater than 50%; if the output is negative, the probability of the outcome is less than 50%.

Compare your result to the actual response in row one above. Did your model correctly classify this observation?

In [ ]:
# INSERT CODE HERE
df_1 = df %>%
  slice(-1)

one = df %>%
  slice(1)

model1 = glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age, data = df_1, family = "binomial")

log_odds = predict(model1, newdata = one, type = "response")

predicted = ifelse(log_odds > 0.5, 1, 0)

actual = one$diabetes


predicted
actual

## yes the model classified the observation correctly.


1 
1

[1] pos
Levels: neg pos

So we just calculated a single iteration of LOOCV. We used 531 rows of our data to fit a model to predict the outcome of the last row.

Below, use a `for` loop to iterate through the rest of your dataset doing the same thing. You will need to:
* Create a data frame `results` with two columns: one named `actual` which holds the true classification for each observation, and one named `predicted`, which should be filled with `NA`s. This is where you'll store the output of your loop.
* Create a loop that runs through each row of your data, pulls that observation out, trains your model on the remaining data, and then tests the fitted model on your test observation.
* Store your model *predictions* ("pos" or "neg" -- not the log-odds) in the `predicted` column of your `results` dataframe

After you run your loop, print the first few lines of `results`.

In [ ]:
# Initialize `results` data frame
results = data.frame(actual = df$diabetes, predicted = NA)

#for loop
for (i in 1:nrow(df)){
    # separate individual observation `i` from the rest of your data
    test_obs = df %>%
      slice(i)
    train_data = df %>%
      slice(-i)

    # train your model
    model = glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age, data = train_data, family = "binomial")

    # test model on hold out observation
    prob = predict(model, newdata = test_obs, type = "response")

    # classify model prediction as "pos" or "neg" and add to `results`
    predicted = ifelse(prob >= 0.5, "pos", "neg")

    results$predicted[i] = predicted

}

head(results)

,actual,predicted
,<fct>,<chr>
1,pos,pos
2,neg,neg
3,neg,neg
4,pos,pos
5,pos,neg
6,pos,pos


Now, calculate the overall error of your model. What proportion of cases were incorrectly classified?

In [ ]:
# INSERT CODE HERE
calc = results %>%
  mutate(actual = ifelse(actual == "pos", 1, 0), predicted = ifelse(predicted == "pos", 1, 0))
accuracy = mean(calc$predicted == calc$actual)
error = 1 - accuracy
error
## 22% were incorrectly classified

[1] 0.2218045

# 3. Compare to `cv.glm` (3 pts)

Now, let's compare this result to the `cv.glm` function. Using the tutorial as a guide, use `cv.glm` to run LOOCV on the data, using the same model (i.e., still using all of the variables to predict diabetes diagnosis).

Note that, because this is a `classification` problem and not a regression problem like in the tutorial, we need to adjust the `cost` argument of `cv.glm`. We can read more about this in the docs:

In [ ]:
#?cv.glm
model = glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age, data = df, family = "binomial")
cost_function <- function(r, i) {
  # Convert probabilities to class labels
  predicted_class <- ifelse(i > 0.5, 1, 0)
  # Calculate misclassification rate
  return(mean(predicted_class != r))
}
cv.diabetes = cv.glm(df, model, cost = cost_function, K=nrow(df))

Here, we see `cost` is defined as:
> "A function of two vector arguments specifying the cost function for the cross-validation. The first argument to cost should correspond to the **observed responses** and the second argument should correspond to the **predicted or fitted responses** from the generalized linear model."

In the example code (scroll to bottom of the docs), we see that the appropriate cost function for a binary classification is

``
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
``

Where `r` is the vector of observed responses (technically "pos" and "neg", but R treats these as 1 and 0 under the hood), and `pi` is the vector of *probabilities* (not log-odds) fit by the model. Thus, this boils down to our error: what proportion of observations were incorrectly classified. You will need to include this code below.

In [ ]:
# INSERT CODE HERE
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
cv.diabetes2 = cv.glm(df, model, cost = cost, K=nrow(df))
cv.diabetes$delta
cv.diabetes2$delta

[1] 0.2218045 0.2221154

[1] 0.2218045 0.2221154

How do your results compare to your manual LOOCV above?

> * *They do equally as well both ending up this a misclassification rate of
 approximately 22%*


# 4. Adjusting K and Reflection (2 pts)

Recall that LOOCV has some drawbacks. In particular, it has quite high *variance* which can lead to poor performance on new test data. We can reduce this variance by increasing K.

Below, re-run your cross validation using `cv.glm` with `k` set to 3, 5, 10, and 15.

In [ ]:
set.seed(1)
#INSERT CODE BELOW

# K = 3
cv.diabetes3 = cv.glm(df, model, cost = cost, K=3)
cv.diabetes3$delta

# K = 5
cv.diabetes5 = cv.glm(df, model, cost = cost, K=5)
cv.diabetes5$delta

# K = 10
cv.diabetes10 = cv.glm(df, model, cost = cost, K=10)
cv.diabetes10$delta

# K = 15
cv.diabetes15 = cv.glm(df, model, cost = cost, K=15)
cv.diabetes15$delta

[1] 0.2105263 0.2168085

[1] 0.2161654 0.2184444

[1] 0.2236842 0.2230942

[1] 0.2274436 0.2281856

#### Reflection

How do your errors compare to your LOOCV error above? How do they change as k increases?
> * *They don't seem significantly different, the lower values of k had lower rates 21% and the higher values had higher rates, some of which would total to 23% iif rounded.*
>
> *  

If you change the random seed above, you'll get slightly different errors. If you were to do the same with your LOOCV above , would you expect to get different results each time? Why or why not?
> * *I would not expect different errors from LOOCV because you leave one out but you do it for all of them so there is no sampling that would give you different errors.*
> *


**DUE:** 5pm March 25, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*
>
>
